In [19]:
%%writefile fetch_data.py
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import os
import re
import json
import selenium

# Initialize Chrome Selenium
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-extensions')
options.add_argument('--disable-gpu')
# user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537'
options.add_argument('user-agent={0}'.format(user_agent))

driver = webdriver.Chrome(options=options) 
# driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options) 
# driver.implicitly_wait(90)
driver.set_page_load_timeout(90)

# Load the URL and get the page source
URL = 'https://liveat678bellflower.com/floor-plans.aspx'
driver.get(URL)

results_container = WebDriverWait(driver, 100).until(
    EC.presence_of_element_located((By.ID, 'par_10617247'))
)

assert type(results_container) == selenium.webdriver.remote.webelement.WebElement, "results_container is a wrong type! Please check."

sections = results_container.find_elements(By.CLASS_NAME, "unit-container")

# Find elements
res = []
for idx, section in enumerate(sections):
    unit_type_str = section.find_element(By.XPATH, '//*[@id="text-area"]/div/div[1]/h2').get_attribute("innerHTML").strip()
    unit_price_str = section.find_element(By.CLASS_NAME, "unit-rent").get_attribute("innerHTML")
    unit_number_str = section.find_element(By.CLASS_NAME, "unit-number").get_attribute("innerHTML")
    unit_sqft_str = section.find_element(By.CLASS_NAME, "unit-sqft").get_attribute("innerHTML")

    unit_sqft = int(re.findall(r'\d+', unit_sqft_str.replace(',', ''))[0])
    unit_number = int(re.findall(r'\d+', unit_number_str)[0])
    unit_price = unit_price_str.replace('$', '').replace(',', '')
    unit_price = int(unit_price[-4:])
    
    assert unit_type_str == 'The Preserve', 'Wrong unit type! Please check.'
    
    _res = {"unit_number": unit_number,
            "unit_price": unit_price,
            "unit_sqft": unit_sqft
            }
    
    res.append(_res)

assert len(res) != 0, "Result list is empty! Please check."

res2 = sorted(res, key=lambda x: (x['unit_price'], x['unit_number']))

# Write out results
with open("unit_prices.json", "w") as f:
    # Dump the list to the file
    json.dump(res2, f)


Overwriting fetch_data.py


## Bellflower apartment price scraper

In [13]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import os
import re
import json

# Initialize Chrome Selenium
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--ignore-certificate-errors')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-extensions')
options.add_argument('--disable-gpu')
# user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537'
options.add_argument('user-agent={0}'.format(user_agent))

driver = webdriver.Chrome(options=options) 
# driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options) 
# driver.set_page_load_timeout(90)
driver.implicitly_wait(90)

In [14]:
URL = 'https://liveat678bellflower.com/floor-plans.aspx'

# Load the URL and get the page source
driver.get(URL)



In [15]:
results_container = WebDriverWait(driver, 100).until(
    # EC.presence_of_element_located((By.CLASS_NAME, "par-units"))
    EC.presence_of_element_located((By.ID, 'par_10617247'))
)

sections = results_container.find_elements(By.CLASS_NAME, "unit-container")

In [39]:
# section.find_element(By.CSS_SELECTOR, "div.unit-container-left > div:nth-child(3)").get_attribute("innerHTML")
section.find_element(By.CSS_SELECTOR, "div.unit-container-left > div:nth-child(4)").get_attribute("innerHTML").replace('Lease Term: ', '')

'12'

In [43]:
s = 'Available: 03/20/2024'
# s = 'NOW'
re.findall(r'(\d{2}/\d{2}/\d{4}|NOW)', s)[0]

'03/20/2024'

In [37]:
# Find elements
res = []
for idx, section in enumerate(sections):
    unit_type_str = section.find_element(By.XPATH, '//*[@id="text-area"]/div/div[1]/h2').get_attribute("innerHTML").strip()
    unit_price_str = section.find_element(By.CLASS_NAME, "unit-rent").get_attribute("innerHTML")
    unit_number_str = section.find_element(By.CLASS_NAME, "unit-number").get_attribute("innerHTML")
    unit_sqft_str = section.find_element(By.CLASS_NAME, "unit-sqft").get_attribute("innerHTML")
    available_date_str = section.find_element(By.CSS_SELECTOR, "div.unit-container-left > div:nth-child(3)").get_attribute("innerHTML")
    lease_term_str = section.find_element(By.CSS_SELECTOR, "div.unit-container-left > div:nth-child(4)").get_attribute("innerHTML")

    unit_sqft = int(re.findall(r'\d+', unit_sqft_str.replace(',', ''))[0])
    unit_number = int(re.findall(r'\d+', unit_number_str)[0])
    unit_price = unit_price_str.replace('$', '').replace(',', '')
    unit_price = int(unit_price[-4:])
    available_date = re.findall(r'\d{2}/\d{2}/\d{4}', 'Available: 04/06/2024')[0]
    lease_term = int(lease_term_str.replace('Lease Term: ', ''))
    
    assert unit_type_str == 'The Preserve', 'Wrong unit type! Please check.'
    
    _res = {"unit_number": unit_number,
            "unit_price": unit_price,
            "unit_sqft": unit_sqft,
            "available_date": available_date,
            "lease_term": lease_term
            }
    
    res.append(_res)
    # res.append(f"{unit_type_str} is renting for ${unit_price:,} ({unit_number_str}, {unit_sqft_str})")
    # print(f"{unit_type_str}: {unit_number_str} is renting for ${unit_price:,} ({unit_sqft_str})")

res2 = sorted(res, key=lambda x: (x['unit_price'], x['unit_number']))
res2    

[{'unit_number': 2,
  'unit_price': 3403,
  'unit_sqft': 1076,
  'available_date': '04/06/2024'},
 {'unit_number': 58,
  'unit_price': 3403,
  'unit_sqft': 1076,
  'available_date': '04/06/2024'},
 {'unit_number': 88,
  'unit_price': 3419,
  'unit_sqft': 1076,
  'available_date': '04/06/2024'},
 {'unit_number': 91,
  'unit_price': 3529,
  'unit_sqft': 1076,
  'available_date': '04/06/2024'}]

In [189]:
import json

res2

with open("my_list.json", "w") as f:
    # Dump the list to the file
    json.dump(res2, f)

## Discord bot

In [4]:
import shutil
shutil.copyfile('latest_prices.json', 'snapshot_prices.json')

'snapshot_prices.json'

In [2]:
# %%writefile discord_bot.py
import discord

bot = discord.Client()

# Event triggered when the bot is ready
@bot.event
async def on_ready():
    print(f'Logged in as {bot.user.name} ({bot.user.id})')
    channel = bot.get_channel(1084864988688154627) 
    await channel.send("{'unit_number': 2, 'unit_price': 3403, 'unit_sqft': 1076, 'available_date': '04/06/2024'}")


bot.run(os.getenv('DISCORD_TOKEN'))


RuntimeError: Cannot close a running event loop